In [1]:
!pip install duckdb

In [2]:
import duckdb

In [3]:
# สร้างการเชื่อมต่อแบบ in-memory
con = duckdb.connect(database=':memory:', read_only=False)

In [4]:
# Query ข้อมูลจาก CSV โดยตรง
query = "SELECT * FROM 'shopping_trends_updated.csv'"

In [6]:
result = con.execute(query).fetchdf()
result

,Order ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,True,Express,True,True,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,True,Express,True,True,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,True,Free Shipping,True,True,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,True,Next Day Air,True,True,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,True,Free Shipping,True,True,31,PayPal,Annually
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3896,40,Female,Hoodie,Clothing,28,Virginia,L,Turquoise,Summer,4.2,False,2-Day Shipping,False,False,32,Venmo,Weekly
3896,3897,52,Female,Backpack,Accessories,49,Iowa,L,White,Spring,4.5,False,Store Pickup,False,False,41,Bank Transfer,Bi-Weekly
3897,3898,46,Female,Belt,Accessories,33,New Jersey,L,Green,Spring,2.9,False,Standard,False,False,24,Venmo,Quarterly
3898,3899,44,Female,Shoes,Footwear,77,Minnesota,S,Brown,Summer,3.8,False,Express,False,False,24,Venmo,Weekly


In [9]:
query = """
-- CTE สำหรับเตรียมข้อมูลตั้งต้น และ "สร้างลำดับที่แน่นอน" ด้วย row_id
WITH enriched_orders AS (
    SELECT
        ROW_NUMBER() OVER() AS row_id,
        "Order ID" AS Order_id, -- แก้ไขจาก " ID" และตั้งชื่อเล่นให้สอดคล้อง
        Category,
        "Purchase Amount (USD)" AS amount,
        Season,
        "Review Rating" AS rating,
        "Shipping Type" AS shipping_type,
        "Payment Method" AS payment_method,
        -- เพิ่มคอลัมน์ที่ขาดไปจากไฟล์ต้นฉบับ
        Age,
        Size,
        Gender,
        Location,
        "Item Purchased" AS item_purchased
    FROM read_csv_auto('shopping_trends_updated.csv')
)

-- Final Output: ใช้ Window Functions พร้อม ORDER BY เพื่อคำนวณแบบสะสม
SELECT
    Order_id,
    Category,
    amount,
    Season,
    shipping_type,
    payment_method,

    -- ค่าเฉลี่ยคะแนนรีวิว "สะสม" ของแต่ละ Category
    ROUND(AVG(rating) OVER (PARTITION BY Category ORDER BY row_id), 2) AS avg_ratinge,

    -- จำนวนครั้ง "สะสม" ของแต่ละประเภทการจัดส่ง
    COUNT(*) OVER (PARTITION BY shipping_type ORDER BY row_id) AS order_count_by_shipping,

    -- จำนวนครั้ง "สะสม" ของแต่ละวิธีการชำระเงิน
    COUNT(*) OVER (PARTITION BY payment_method ORDER BY row_id) AS count_by_payment,

    -- จำนวนครั้ง "สะสม" ของแต่ละฤดู
    COUNT(*) OVER (PARTITION BY Season ORDER BY row_id) AS order_count_by_season,

    -- ลำดับการสั่งซื้อในแต่ละ Category
    ROW_NUMBER() OVER (PARTITION BY Category ORDER BY Order_id) AS order_sequence_by_category,

    -- จัดอันดับยอดซื้อในแต่ละ Category (ยอดสูงสุดได้ Rank 1)
    RANK() OVER (PARTITION BY Category ORDER BY amount DESC) AS rank_purchase_amount_by_category,

    -- ยอดซื้อสะสมในแต่ละ Category
    SUM(amount) OVER (
        PARTITION BY Category
        ORDER BY row_id
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS cumulative_purchase_amount_by_category,

    -- สินค้าที่ซื้อก่อนหน้าใน Category เดียวกัน
    LAG(item_purchased, 1, 'N/A') OVER (
        PARTITION BY Category
        ORDER BY row_id
    ) AS previous_item_in_category,

    -- นับอายุ 20–25 แบบสะสม (จะแสดงค่าเฉพาะแถวที่เข้าเงื่อนไข)
    CASE
        WHEN Age BETWEEN 20 AND 25 THEN
            COUNT(*) FILTER (WHERE Age BETWEEN 20 AND 25)
            OVER (ORDER BY row_id ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
    END AS order_count_age_20_25,

    -- นับลำดับการสั่งซื้อตาม Size และ Gender
    ROW_NUMBER() OVER (PARTITION BY Size, Gender ORDER BY row_id) AS order_sequence_by_size_gender,


    -- % การสั่งซื้อในแต่ละ Location เทียบกับทั้งหมด
    COUNT(*) OVER (PARTITION BY Location) * 100.0 / COUNT(*) OVER () AS percent_order_by_location

FROM
    enriched_orders -- ย้าย FROM มาไว้ตรงนี้
ORDER BY
    row_id; -- เรียงผลลัพธ์ทั้งหมดตาม row_id เพื่อให้เห็นการสะสมที่ชัดเจน

"""

window_analysis = con.execute(query).fetchdf()

window_analysis

,Order_id,Category,amount,Season,shipping_type,payment_method,avg_ratinge,order_count_by_shipping,count_by_payment,order_count_by_season,order_sequence_by_category,rank_purchase_amount_by_category,cumulative_purchase_amount_by_category,previous_item_in_category,order_count_age_20_25,order_sequence_by_size_gender,percent_order_by_location
0,1,Clothing,53,Winter,Express,Venmo,3.10,1,1,1,1,1006,53.0,N/A,<NA>,1,2.025641
1,2,Clothing,64,Winter,Express,Cash,3.10,2,1,2,2,778,117.0,Blouse,<NA>,2,1.974359
2,3,Clothing,73,Spring,Free Shipping,Credit Card,3.10,1,1,1,3,589,190.0,Sweater,<NA>,1,1.846154
3,4,Footwear,90,Spring,Next Day Air,PayPal,3.50,1,1,2,1,78,90.0,N/A,1,1,1.615385
4,5,Clothing,49,Spring,Free Shipping,PayPal,3.00,2,2,3,4,1094,239.0,Jeans,<NA>,2,1.897436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3896,Clothing,28,Summer,2-Day Shipping,Venmo,3.72,627,631,954,1737,1537,104264.0,Skirt,<NA>,335,1.974359
3896,3897,Accessories,49,Spring,Store Pickup,Bank Transfer,3.77,649,612,997,1238,778,74086.0,Hat,<NA>,336,1.769231
3897,3898,Accessories,33,Spring,Standard,Venmo,3.77,654,632,998,1239,1017,74119.0,Backpack,<NA>,337,1.717949
3898,3899,Footwear,77,Summer,Express,Venmo,3.79,646,633,955,599,173,36093.0,Sneakers,<NA>,187,2.256410
